In [156]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import time
from matplotlib import pyplot as plt
from census import Census
from scipy.stats import linregress



# Import API key
from api_keys import geoapify_key

In [157]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


In [158]:
#%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600 
)
# Display the map plot

# Display the map plot
map_plot_1
# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

In [159]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather_df = city_data_df.copy()

ideal_weather_df

filter1 = city_data_df["Cloudiness"] == 0
filter2 = city_data_df["Wind Speed"] <= 4.5

ideal_weather_df.where(filter1 & filter2, inplace=True)

ideal_weather_df.dropna(inplace=True)


# Display sample data
ideal_weather_df.head(15)
# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45.0,kapaa,22.0752,-159.3190,22.99,84.0,0.0,3.60,US,1.666108e+09
51,51.0,hilo,19.7297,-155.0900,26.27,83.0,0.0,2.57,US,1.666108e+09
63,63.0,banda,25.4833,80.3333,24.62,52.0,0.0,2.68,IN,1.666108e+09
81,81.0,makakilo city,21.3469,-158.0858,21.66,81.0,0.0,2.57,US,1.666108e+09
90,90.0,saint george,37.1041,-113.5841,13.33,44.0,0.0,1.54,US,1.666108e+09
96,96.0,zhangye,38.9342,100.4517,11.11,16.0,0.0,1.67,CN,1.666108e+09
115,115.0,cedar city,37.6775,-113.0619,14.11,49.0,0.0,0.00,US,1.666108e+09
121,121.0,lichuan,30.3000,108.8500,9.72,77.0,0.0,1.10,CN,1.666108e+09
128,128.0,half moon bay,37.4636,-122.4286,19.71,86.0,0.0,0.00,US,1.666108e+09
141,141.0,brenham,30.1669,-96.3977,14.47,40.0,0.0,3.09,US,1.666108e+09


In [160]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
# create a data frame from cities, lat, and temp
hotel_df = ideal_weather_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]].head()
#otel_df.head()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# YOUR CODE HERE
hotel_df.head()
# Display sample data
# YOUR CODE HERE

,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84.0,
51,hilo,US,19.7297,-155.0900,83.0,
63,banda,IN,25.4833,80.3333,52.0,
81,makakilo city,US,21.3469,-158.0858,81.0,
90,saint george,US,37.1041,-113.5841,44.0,


In [163]:
# Set parameters to search for a hotel
#latitude = -31.9523
#longitude = 115.8613
radius = 10000
categories = "accommodation.hotel"

    

params = {
    "categories":categories,
    "apiKey":geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    # YOUR CODE HERE
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
       

        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
banda - nearest hotel: No hotel found
makakilo city - nearest hotel: No hotel found
saint george - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84.0,No hotel found
51,hilo,US,19.7297,-155.0900,83.0,No hotel found
63,banda,IN,25.4833,80.3333,52.0,No hotel found
81,makakilo city,US,21.3469,-158.0858,81.0,No hotel found
90,saint george,US,37.1041,-113.5841,44.0,No hotel found


In [162]:
#%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
Hotel_map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Distance",
    scale = 0.5,
    color = "City"
)

# Display the map plot
Hotel_map_2
# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)